In [5]:
from gym_match3.envs import Match3Env
import gym
import matplotlib.pyplot as plt
# import pickle
from matplotlib import style
import numpy as np
#from gym_match3.envs.levels import Match3Levels, Level
#env = Match3Env()
#obs, reward, done, info = env.step(0) 
import torch
import cv2
from PIL import Image
from gym_match3.envs import Match3Env
from gym_match3.envs.levels import LEVELS #  default levels
from gym_match3.envs.levels import Match3Levels, Level

In [ ]:
#env = Match3Env() 
#env.observation_space
#env.get_available_actions()   #查看action對應的動作

In [48]:
def one_hot_encoding(obs):
    
    obs = obs.reshape(obs.shape[1],obs.shape[2]).astype(int)
    
    grid_onehot = np.zeros(shape=(env.n_shapes, obs.shape[0], obs.shape[1]))
    table = {i:i for i in range(0,10)}
    
    for i in range(obs.shape[0]):
        for j in range(obs.shape[1]):
            grid_element = obs[i][j]
            grid_onehot[table[grid_element]][i][j]=1
            
    return grid_onehot

In [54]:
def plot_obs(observation=None):
    
    d = {0: (255, 255, 255),   #白色
     1: (0, 255, 255),      #黃色
     2: (0, 0, 255) ,      #紅色
     3: (230,224,176) ,   #灰藍色 
     4: (0,0,0), #黑色  
     5: (100,100,100),
        6:(150,150,150),
        7:(200,200,200),
        8:(200,150,250),
        9:(250,200,150),
        10:(150,200,250)}              
    
    
    background = np.zeros((observation.shape[1],observation.shape[2],3), dtype=np.uint8) #黑色的背景

    for color in range(env.n_shapes):
        result = np.where(one_hot_encoding(observation)[color] == 1)
        
        listOfCoordinates= list(zip(result[0], result[1]))
        
        for cord in listOfCoordinates:
            background[cord] = d[color]     
            
    img = Image.fromarray(background, 'RGB')
     
    
    #img = img.resize((500, 800)) 
    cv2.imshow("image", np.array(img))
    
    cv2.waitKey(200)

In [58]:
from gym_match3.envs import Match3Env
import gym
from PIL import Image
import cv2
import numpy as np
   

def Getlevels(WidthAndHeight,shapes):
    LEVELS = [Level(WidthAndHeight,WidthAndHeight,shapes, np.zeros((WidthAndHeight,WidthAndHeight)).tolist())]
    return LEVELS
    


env = Match3Env(levels=Match3Levels(Getlevels(8,5)))

for i_episode in range(1): #玩一次遊戲
    
    #obscont = 1    
    state = env.reset() 
    while True:
    
        
        action = env.action_space.sample()

        print('---------------')


        observation, reward, done, info = env.step(action)

        

        print(observation.shape)

        print(observation)
        
        plot_obs(observation)

        print('the swap of coordinate is: ',list(env.get_available_actions()[action]))
        #print('reward:', reward)

        if done:
            #print("Episode finished after {} timesteps",obs)
            #print(obscont)
            break
env.close()

---------------
(1, 8, 8)
[[[4. 2. 4. 2. 2. 4. 0. 1.]
  [4. 3. 3. 0. 0. 1. 1. 4.]
  [3. 0. 3. 4. 4. 0. 3. 3.]
  [0. 2. 4. 3. 0. 3. 2. 3.]
  [3. 4. 0. 1. 1. 0. 1. 4.]
  [2. 0. 1. 3. 1. 0. 2. 0.]
  [0. 3. 0. 4. 0. 4. 0. 4.]
  [3. 2. 3. 2. 4. 3. 0. 1.]]]
the swap of coordinate is:  [(4, 6), (3, 6)]
---------------
(1, 8, 8)
[[[4. 2. 4. 2. 2. 4. 0. 1.]
  [4. 3. 3. 0. 0. 1. 1. 4.]
  [3. 0. 3. 4. 4. 0. 3. 3.]
  [0. 2. 4. 3. 0. 3. 2. 3.]
  [3. 4. 0. 1. 1. 0. 1. 4.]
  [2. 0. 1. 3. 1. 0. 2. 0.]
  [0. 3. 0. 4. 0. 4. 0. 4.]
  [3. 2. 3. 2. 4. 3. 0. 1.]]]
the swap of coordinate is:  [(4, 2), (3, 2)]
---------------
(1, 8, 8)
[[[4. 2. 4. 2. 2. 4. 0. 1.]
  [4. 3. 3. 0. 0. 1. 1. 4.]
  [3. 0. 3. 4. 4. 0. 3. 3.]
  [0. 2. 4. 3. 0. 3. 2. 3.]
  [3. 4. 0. 1. 1. 0. 1. 4.]
  [2. 0. 1. 3. 1. 0. 2. 0.]
  [0. 3. 0. 4. 0. 4. 0. 4.]
  [3. 2. 3. 2. 4. 3. 0. 1.]]]
the swap of coordinate is:  [(3, 4), (2, 4)]
---------------
(1, 8, 8)
[[[4. 2. 4. 2. 2. 4. 0. 1.]
  [4. 3. 3. 0. 0. 1. 1. 4.]
  [3. 0. 3. 4. 4. 0. 3. 3.]

the swap of coordinate is:  [(7, 4), (6, 4)]
---------------
(1, 8, 8)
[[[1. 2. 2. 0. 2. 4. 2. 3.]
  [2. 4. 3. 4. 4. 2. 3. 2.]
  [2. 4. 4. 1. 0. 4. 2. 1.]
  [4. 2. 1. 2. 2. 1. 0. 3.]
  [4. 3. 4. 2. 3. 3. 1. 4.]
  [3. 2. 4. 3. 1. 0. 3. 0.]
  [2. 0. 0. 3. 3. 2. 2. 4.]
  [0. 2. 0. 2. 1. 3. 1. 1.]]]
the swap of coordinate is:  [(4, 5), (4, 4)]
---------------
(1, 8, 8)
[[[1. 2. 2. 0. 2. 4. 2. 3.]
  [2. 4. 3. 4. 4. 2. 3. 2.]
  [2. 4. 4. 1. 0. 4. 2. 1.]
  [4. 2. 1. 2. 2. 1. 0. 3.]
  [4. 3. 4. 2. 3. 3. 1. 4.]
  [3. 2. 4. 3. 1. 0. 3. 0.]
  [2. 0. 0. 3. 3. 2. 2. 4.]
  [0. 2. 0. 2. 1. 3. 1. 1.]]]
the swap of coordinate is:  [(4, 2), (4, 3)]
---------------
(1, 8, 8)
[[[1. 2. 2. 0. 2. 4. 2. 3.]
  [2. 4. 3. 4. 4. 2. 3. 2.]
  [2. 4. 4. 1. 0. 4. 2. 1.]
  [4. 2. 1. 2. 2. 1. 0. 3.]
  [4. 3. 4. 2. 3. 3. 1. 4.]
  [3. 2. 4. 3. 1. 0. 3. 0.]
  [2. 0. 0. 3. 3. 2. 2. 4.]
  [0. 2. 0. 2. 1. 3. 1. 1.]]]
the swap of coordinate is:  [(1, 2), (1, 3)]
---------------
(1, 8, 8)
[[[1. 2. 2. 0. 2. 4. 2. 3.]
  [2. 4. 3

(1, 8, 8)
[[[0. 1. 4. 3. 0. 3. 0. 0.]
  [1. 0. 0. 3. 2. 0. 3. 2.]
  [2. 4. 3. 2. 0. 4. 3. 3.]
  [4. 2. 1. 1. 2. 0. 0. 1.]
  [4. 3. 4. 4. 0. 4. 0. 4.]
  [3. 2. 4. 0. 2. 1. 1. 0.]
  [2. 0. 0. 1. 1. 0. 2. 4.]
  [0. 2. 0. 3. 1. 2. 1. 1.]]]
the swap of coordinate is:  [(7, 4), (7, 3)]
---------------
(1, 8, 8)
[[[0. 1. 4. 3. 0. 3. 0. 0.]
  [1. 0. 0. 3. 2. 0. 3. 2.]
  [2. 4. 3. 2. 0. 4. 3. 3.]
  [4. 2. 1. 1. 2. 0. 0. 1.]
  [4. 3. 4. 4. 0. 4. 0. 4.]
  [3. 2. 4. 0. 2. 1. 1. 0.]
  [2. 0. 0. 1. 1. 0. 2. 4.]
  [0. 2. 0. 3. 1. 2. 1. 1.]]]
the swap of coordinate is:  [(1, 5), (2, 5)]
---------------
(1, 8, 8)
[[[0. 1. 4. 3. 0. 3. 0. 0.]
  [1. 0. 0. 3. 2. 0. 3. 2.]
  [2. 4. 3. 2. 0. 4. 3. 3.]
  [4. 2. 1. 1. 2. 0. 0. 1.]
  [4. 3. 4. 4. 0. 4. 0. 4.]
  [3. 2. 4. 0. 2. 1. 1. 0.]
  [2. 0. 0. 1. 1. 0. 2. 4.]
  [0. 2. 0. 3. 1. 2. 1. 1.]]]
the swap of coordinate is:  [(1, 2), (0, 2)]
---------------
(1, 8, 8)
[[[0. 1. 4. 3. 0. 3. 0. 0.]
  [1. 0. 0. 3. 2. 0. 3. 2.]
  [2. 4. 3. 2. 0. 4. 3. 3.]
  [4. 2. 1. 1. 

the swap of coordinate is:  [(1, 1), (2, 1)]
---------------
(1, 8, 8)
[[[0. 0. 3. 0. 1. 4. 0. 0.]
  [1. 1. 0. 0. 3. 2. 4. 2.]
  [2. 4. 4. 3. 0. 2. 0. 3.]
  [4. 2. 3. 2. 1. 0. 3. 1.]
  [4. 3. 4. 4. 2. 4. 3. 4.]
  [3. 2. 0. 4. 3. 4. 0. 0.]
  [2. 0. 3. 1. 4. 0. 2. 4.]
  [0. 2. 4. 3. 0. 2. 1. 1.]]]
the swap of coordinate is:  [(4, 2), (3, 2)]
---------------
(1, 8, 8)
[[[2. 4. 3. 0. 1. 4. 0. 0.]
  [0. 0. 3. 0. 3. 2. 4. 2.]
  [1. 1. 0. 3. 0. 2. 0. 3.]
  [2. 2. 3. 2. 1. 0. 3. 1.]
  [4. 3. 4. 4. 2. 4. 3. 4.]
  [3. 2. 0. 4. 3. 4. 0. 0.]
  [2. 0. 3. 1. 4. 0. 2. 4.]
  [0. 2. 4. 3. 0. 2. 1. 1.]]]
the swap of coordinate is:  [(3, 0), (2, 0)]
---------------
(1, 8, 8)
[[[2. 4. 3. 0. 1. 4. 0. 0.]
  [0. 0. 3. 0. 3. 2. 4. 2.]
  [1. 1. 0. 3. 0. 2. 0. 3.]
  [2. 2. 3. 2. 1. 0. 3. 1.]
  [4. 3. 4. 4. 2. 4. 3. 4.]
  [3. 2. 0. 4. 3. 4. 0. 0.]
  [2. 0. 3. 1. 4. 0. 2. 4.]
  [0. 2. 4. 3. 0. 2. 1. 1.]]]
the swap of coordinate is:  [(0, 6), (1, 6)]
---------------
(1, 8, 8)
[[[2. 4. 3. 0. 1. 4. 0. 0.]
  [0. 0. 3